In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics.pairwise import cosine_similarity

# Baca dataset
ratings = pd.read_csv("https://s3-us-west-2.amazonaws.com/recommender-tutorial/ratings.csv")

# Fungsi training model sederhana
def train_recommendation_model(ratings):
    # Buat matriks interaksi pengguna-film
    user_movie_matrix = ratings.pivot_table(
        index='userId', 
        columns='movieId', 
        values='rating', 
        fill_value=0
    )
    
    # Hitung similarity antar pengguna
    user_similarity = cosine_similarity(user_movie_matrix)
    
    return user_movie_matrix, user_similarity

# Fungsi prediksi rekomendasi
def predict_recommendations(user_id, user_movie_matrix, user_similarity, top_n=3):
    # Cari film yang belum dinilai user
    user_ratings = user_movie_matrix.loc[user_id]
    unrated_movies = user_ratings[user_ratings == 0].index
    
    # Hitung skor rekomendasi
    recommendations = {}
    for movie in unrated_movies:
        movie_score = user_similarity[user_movie_matrix.index.get_loc(user_id)].dot(user_movie_matrix[movie])
        recommendations[movie] = movie_score
    
    # Ambil top N rekomendasi
    top_recommendations = sorted(
        recommendations.items(), 
        key=lambda x: x[1], 
        reverse=True
    )[:top_n]
    
    return [rec[0] for rec in top_recommendations]

# Proses utama
def main():
    # Split data training dan testing
    train_data, test_data = train_test_split(ratings, test_size=0.2, random_state=42)
    
    # Training model
    user_movie_matrix, user_similarity = train_recommendation_model(train_data)
    
    # Contoh prediksi untuk satu user
    sample_user = user_movie_matrix.index[0]
    recommendations = predict_recommendations(
        sample_user, 
        user_movie_matrix, 
        user_similarity
    )
    
    print(f"Rekomendasi film untuk User {sample_user}:")
    for idx, movie in enumerate(recommendations, 1):
        print(f"{idx}. Film ID: {movie}")

# Jalankan program
main()

Rekomendasi film untuk User 1:
1. Film ID: 318
2. Film ID: 589
3. Film ID: 1198
